In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install timm
!pip install transformers timm ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
def test_model(model_path, model_constructor, model_name="model"):


    model = model_constructor()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    preds = []
    trues = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            preds.extend(predicted.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    acc  = accuracy_score(trues, preds)
    prec = precision_score(trues, preds, average="weighted", zero_division=0)
    rec  = recall_score(trues, preds, average="weighted", zero_division=0)
    f1   = f1_score(trues, preds,     average="weighted", zero_division=0)

    print(f"Test Accuracy : {acc:.4f}")
    print(f"Test Precision: {prec:.4f}")
    print(f"Test Recall   : {rec:.4f}")
    print(f"Test F1 Score : {f1:.4f}")

    return acc, prec, rec, f1

In [ ]:

# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Scaled/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Scaled/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Scaled/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
 Copied data to local Colab disk.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 640467232 bytes (611 MiB)

Extracting archive: /content/test.7z
--
Path = /content/test.7z
Type = 7z
Physical Size = 640467232
Headers Size = 97480
Method = LZMA2:24 BCJ
Solid = +
Blocks = 1

  0%      0% 43 - DL_Temp/test/ai/042_sdv4_00106_resize.jpg                                                     0% 85 - DL_Temp/te

In [ ]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path = "/content/DL_Temp/train"
val_path  = "/content/DL_Temp/validation"
test_path = "/content/DL_Temp/test"

train_data = datasets.ImageFolder(train_path, transform=transform)
val_data  = datasets.ImageFolder(val_path,  transform=transform)
test_data = datasets.ImageFolder(test_path,  transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader  = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data,  batch_size=32, shuffle=False)

class_names = train_data.classes
num_classes = len(class_names)
num_classes

2

In [ ]:
test_model(
    model_path="/content/drive/MyDrive/resnet50_standard.pth",
    model_constructor=lambda: timm.create_model("resnet50", pretrained=False, num_classes=2),
    model_name="ResNet50"
)

Test Accuracy : 0.8297
Test Precision: 0.8528
Test Recall   : 0.8297
Test F1 Score : 0.8268


(0.8296925003478502, 0.852756538812608, 0.8296925003478502, 0.8268138472099685)

In [ ]:
test_model(
    model_path="/content/drive/MyDrive/shufflenetv2.pth",
    model_constructor=lambda: shufflenet_v2_x1_0(weights=None, num_classes=2),
    model_name="ShuffleNetV2"
)

Test Accuracy : 0.8924
Test Precision: 0.8965
Test Recall   : 0.8924
Test F1 Score : 0.8922


(0.8924446918046472,
 0.8965172814251066,
 0.8924446918046472,
 0.8921566044160958)